### Denoising Autoencoders And Where To Find Them

Today we're going to train deep autoencoders and deploy them to faces and search for similar images.

Our new test subjects are human faces from the [lfw dataset](http://vis-www.cs.umass.edu/lfw/).

In [ ]:
import numpy as np
from lfw_dataset import fetch_lfw_dataset
from sklearn.model_selection import train_test_split
X, attr = fetch_lfw_dataset(use_raw=True,dimx=38,dimy=38)
X = X.transpose([0,3,1,2]).astype('float32') / 256.0

img_shape = X.shape[1:]
print("Image shape:",img_shape)

X_train, X_test = train_test_split(X, test_size=0.1,random_state=42)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.title('sample image')
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(X[i].transpose([1,2,0]))

print("X shape:",X.shape)
print("attr shape:",attr.shape)

### Autoencoder architecture

Let's design autoencoder as a single lasagne network, going from input image through bottleneck into the reconstructed image.

<img src="http://nghiaho.com/wp-content/uploads/2012/12/autoencoder_network1.png" width=640px>



## First step: PCA

Principial Component Analysis is a popular dimensionality reduction method. 

Under the hood, PCA attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;

In geometric terms, we want to find d axes along which most of variance occurs. The "natural" axes, if you wish.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/PCA_fish.png/256px-PCA_fish.png)


PCA can also be seen as a special case of an autoencoder.

* __Encoder__: X -> Dense(d units) -> code
* __Decoder__: code -> Dense(m units) -> X

Where Dense is a fully-connected layer with linear activaton:   $f(X) = W \cdot X + \vec b $


Note: the bias term in those layers is responsible for "centering" the matrix i.e. substracting mean.

In [ ]:
import theano,theano.tensor as T
import lasagne,lasagne.layers as L
def build_pca_autoencoder(img_shape,code_size=32):
    """
    Here we define a simple linear autoencoder as described above.
    We also reshape decoded data to be compatible with image shapes
    """
    inp = L.InputLayer((None,)+img_shape)
    enc = L.DenseLayer(inp,code_size,nonlinearity=None)
    
    dec = L.DenseLayer(enc,np.prod(img_shape),nonlinearity=None)  #actual decoder, height*width*3 units
    dec = L.ReshapeLayer(dec,(-1,)+img_shape)
    
    return inp,enc,dec

Meld them together into one model

In [ ]:
inp, encoder, decoder = build_pca_autoencoder(img_shape,code_size=32)

code,reconstruction = L.get_output([encoder,decoder])
loss = T.mean((inp.input_var - reconstruction)**2)
updates = lasagne.updates.adamax(loss,L.get_all_params(decoder,trainable=True))

train_step = theano.function([inp.input_var],loss,updates=updates,allow_input_downcast=True)
compute_loss = theano.function([inp.input_var],loss,allow_input_downcast=True)
encode_decode = theano.function([inp.input_var],[code,reconstruction],allow_input_downcast=True)

### Train the model

As usual, iterate minibatches of data and call train_step, then evaluate loss on validation data.

__Note to py2 users:__ you can safely drop `flush=True` from any code below.

In [ ]:
from tqdm import tqdm
import sys
def iterate_minibatches(data, batch_size = 32,verbose = True):
    indices = np.random.permutation(np.arange(len(data)))
    batches = range(0, len(data), batch_size)
    if verbose: 
        batches = tqdm(batches)
    return (data[indices[start_idx:start_idx + batch_size]] for start_idx in batches)

In [ ]:
for epoch in range(32):
    losses = []
    for x_batch in iterate_minibatches(X_train,batch_size=50):
        losses.append(train_step(x_batch))
    print("#%i, Train loss: %.7f"%(epoch+1,np.mean(losses)),flush=True)
    
    val_losses = list(map(compute_loss,iterate_minibatches(X_test,verbose=False)))
    print("#%i, Test loss: %.7f"%(epoch+1,np.mean(val_losses)),flush=True)
    

In [ ]:
def visualize(img,encoder,decoder):
    """Draws original, encoded and decoded images"""
    code,reco = encode_decode(img[None])

    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img.transpose([1,2,0]))

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco[0].transpose([1,2,0]).clip(0,1))
    plt.show()


In [ ]:
score = np.mean(list(map(compute_loss,iterate_minibatches(X_test,verbose=False))))
print("Final MSE:",score)

for i in range(5):
    img = X_test[i]
    visualize(img,encoder,decoder)


### Going deeper

PCA is neat but surely we can do better. This time we want you to build a deep autoencoder by... stacking more layers.

In particular, your encoder and decoder should be at least 3 layers deep each. You can use any nonlinearity you want and any number of hidden units in non-bottleneck layers provided you can actually afford training it.

![layers](https://pbs.twimg.com/media/CYggEo-VAAACg_n.png:small)

A few sanity checks:
* There shouldn't be any hidden layer smaller than bottleneck (encoder output).
* Don't forget to insert nonlinearities between intermediate dense layers.
* Convolutional layers are allowed but not required. To undo convolution use L.Deconv2D, pooling - L.UpSampling2D.
* Adding activation after bottleneck is allowed, but not strictly necessary.

In [ ]:
def build_deep_autoencoder(img_shape,code_size=32):
    """PCA's deeper brother. See instructions above"""
    C,H,W = img_shape
    
    inp = L.InputLayer((None,)+img_shape)
    
    <Your code: define encoder as per instructions above>
    encoder = <...>
    
    <Your code: define encoder as per instructions above>
    decoder = <...>
    
    return inp,encoder,decoder

In [ ]:
#Check autoencoder shapes along different code_sizes
get_dim = lambda layer: np.prod(layer.output_shape[1:])
for code_size in [1,8,32,128,512,1024]:
    _,encoder,decoder = build_deep_autoencoder(img_shape,code_size=code_size)
    print("Testing code size %i" % code_size)
    assert encoder.output_shape[1:]==(code_size,),"encoder must output a code of required size"
    assert decoder.output_shape[1:]==img_shape,   "decoder must output an image of valid shape"
    assert len(L.get_all_params(decoder))>=6,       "encoder must contain at least 3 dense layers"
    
    for layer in L.get_all_layers(decoder):
        assert get_dim(layer) >= code_size, "Encoder layer %s is smaller than bottleneck (%i units)"%(layer.name,get_dim(layer))

print("All tests passed!")

__Hint:__ if you're getting "Encoder layer is smaller than bottleneck" error, use code_size when defining intermediate layers. 

For example, such layer may have code_size*2 units.

In [ ]:
inp, encoder, decoder = build_pca_autoencoder(img_shape,code_size=32)

code,reconstruction = L.get_output([encoder,decoder])
loss = T.mean((inp.input_var - reconstruction)**2)
updates = lasagne.updates.adamax(loss,L.get_all_params(decoder,trainable=True))

train_step = theano.function([inp.input_var],loss,updates=updates,allow_input_downcast=True)
compute_loss = theano.function([inp.input_var],loss,allow_input_downcast=True)
encode_decode = theano.function([inp.input_var],[code,reconstruction],allow_input_downcast=True)

Training may take long, it's okay.

In [ ]:
for epoch in range(32):
    losses = []
    for x_batch in iterate_minibatches(X_train,batch_size=50):
        losses.append(train_step(x_batch))
    print("#%i, Train loss: %.7f"%(epoch+1,np.mean(losses)),flush=True)
    
    val_losses = list(map(compute_loss,iterate_minibatches(X_test,verbose=False)))
    print("#%i, Test loss: %.7f"%(epoch+1,np.mean(val_losses)),flush=True)
    

In [ ]:
reconstruction_mse = np.mean(list(map(compute_loss,iterate_minibatches(X_test,verbose=False))))
assert reconstruction_mse <= 0.005, "Compression is too lossy. See tips below."
assert len(encoder.output_shape)==2 and encoder.output_shape[1]==32, "Make sure encoder has code_size units"
print("Final MSE:", reconstruction_mse)
for i in range(5):
    img = X_test[i]
    visualize(img,encoder,decoder)

__Tips:__ If you keep getting "Compression to lossy" error, there's a few things you might try:

* Make sure it converged. Some architectures need way more than 32 epochs to converge. They may fluctuate a lot, but eventually they're going to get good enough to pass. You may train your network for as long as you want.

* Complexity. If you already have, like, 152 layers and still not passing threshold, you may wish to start from something simpler instead and go in small incremental steps.

* Architecture. You can use any combination of layers (including convolutions, normalization, etc) as long as __encoder output only stores 32 numbers per training object__. 

A cunning learner can circumvent this last limitation by using some manual encoding strategy, but he is strongly recommended to avoid that.

## Denoising AutoEncoder

Let's now make our model into a denoising autoencoder.

We'll keep your model architecture, but change the way it trains. In particular, we'll corrupt it's input data randomly before each epoch.

There are many strategies to apply noise. We'll implement two popular one: adding gaussian noise and using dropout.

In [ ]:
def apply_gaussian_noise(X,sigma=0.1):
    """
    adds noise from normal distribution with standard deviation sigma
    :param X: image tensor of shape [batch,height,width,3]
    """
    
    <your code here>
        
    return X + noise
    

In [ ]:
#noise tests
theoretical_std = (X[:100].std()**2 + 0.5**2)**.5
our_std = apply_gaussian_noise(X[:100],sigma=0.5).std()
assert abs(theoretical_std - our_std) < 0.01, "Standard deviation does not match it's required value. Make sure you use sigma as std."
assert abs(apply_gaussian_noise(X[:100],sigma=0.5).mean() - X[:100].mean()) < 0.01, "Mean has changed. Please add zero-mean noise"

In [ ]:
plt.subplot(1,4,1)
plt.imshow(X[0])
plt.subplot(1,4,2)
plt.imshow(apply_gaussian_noise(X[:1],sigma=0.01)[0])
plt.subplot(1,4,3)
plt.imshow(apply_gaussian_noise(X[:1],sigma=0.1)[0])
plt.subplot(1,4,4)
plt.imshow(apply_gaussian_noise(X[:1],sigma=0.5)[0])

In [ ]:
inp, encoder, decoder = build_pca_autoencoder(img_shape,code_size=32)

code,reconstruction = L.get_output([encoder,decoder])
loss = T.mean((inp.input_var - reconstruction)**2)
updates = lasagne.updates.adamax(loss,L.get_all_params(decoder,trainable=True))

train_step = theano.function([inp.input_var],loss,updates=updates,allow_input_downcast=True)
compute_loss = theano.function([inp.input_var],loss,allow_input_downcast=True)
encode_decode = theano.function([inp.input_var],[code,reconstruction],allow_input_downcast=True)

In [ ]:
for epoch in range(50):
    print("Epoch %i/50, Generating corrupted samples..."%epoch)
    X_train_noise = apply_gaussian_noise(X_train)
    X_test_noise = apply_gaussian_noise(X_test)
    
    losses = []
    for x_batch in iterate_minibatches(X_train_noise,batch_size=50):
        losses.append(train_step(x_batch))
    print("#%i, Train loss: %.7f"%(epoch+1,np.mean(losses)),flush=True)
    
    val_losses = list(map(compute_loss,iterate_minibatches(X_test_noise,verbose=False)))
    print("#%i, Test loss: %.7f"%(epoch+1,np.mean(val_losses)),flush=True)


__Note:__ if it hasn't yet converged, increase the number of iterations.

__Bonus:__ replace gaussian noise with masking random rectangles on image.

In [ ]:
reconstruction_mse = np.mean(list(map(compute_loss,iterate_minibatches(X_test,verbose=False))))
print("Final MSE:", reconstruction_mse)
for i in range(5):
    img = X_test[i]
    visualize(img,encoder,decoder)

### Image retrieval with autoencoders

So we've just trained a network that converts image into itself imperfectly. This task is not that useful in and of itself, but it has a number of awesome side-effects. Let's see it in action.

First thing we can do is image retrieval aka image search. We we give it an image and find similar images in latent space. 

To speed up retrieval process, we shall use Locality-Sensitive Hashing on top of encoded vectors. We'll use scikit-learn's implementation for simplicity. In practical scenario, you may want to use [specialized libraries](https://erikbern.com/2015/07/04/benchmark-of-approximate-nearest-neighbor-libraries.html) for better performance and customization.

In [ ]:
#compile function that encodes batch of images into a batch of vector[batch,code_size]
encode = theano.function([inp.input_var],code,allow_input_downcast=True)
#... and another function that casts those codes back into images
codes = T.matrix("codes")
decode = theano.function([codes],L.get_output(decoder,{encoder:codes}))


In [ ]:
images = X_train
codes = <encode all images>
assert len(codes) == len(images)

In [ ]:
from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=50).fit(codes)

In [ ]:
def get_similar(image, n_neighbors=5):
    assert image.ndim==3,"image must be [batch,height,width,3]"

    code = <encode image>
    
    (distances,),(idx,) = lshf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,images[idx]

In [ ]:
def show_similar(image):
    
    distances,neighbors = get_similar(image,n_neighbors=11)
    
    plt.figure(figsize=[8,6])
    plt.subplot(3,4,1)
    plt.imshow(image)
    plt.title("Original image")
    
    for i in range(11):
        plt.subplot(3,4,i+2)
        plt.imshow(neighbors[i])
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:
#smiles
show_similar(X_test[2])

In [ ]:
#ethnicity
show_similar(X_test[500])

In [ ]:
#glasses
show_similar(X_test[66])

## Bonus: cheap image morphing


In [ ]:

for _ in range(5):
    image1,image2 = X_test[np.random.randint(0,len(X_test),size=2)]

    code1, code2 = encode(np.stack([image1,image2]))

    plt.figure(figsize=[10,4])
    for i,a in enumerate(np.linspace(0,1,num=7)):

        output_code = code1*(1-a) + code2*(a)
        output_image = decode(output_code[None])[0]

        plt.subplot(1,7,i+1)
        plt.imshow(output_image.transpose([1,2,0]).clip(0,1))
        plt.title("a=%.2f"%a)
        
    plt.show()

Of course there's a lot more you can do with autoencoders.

If you want to generate images from scratch, however, we recommend you our honor track seminar about generative adversarial networks.